In [23]:
# Part 1: Initialization and Loading PGN Files

# Import required libraries
import chess
import chess.engine
import chess.pgn
import subprocess
import os
import json
import csv
from IPython.display import display

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}

# Initialize Stockfish engine
print("Initializing Stockfish engine...")
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
print("Stockfish engine initialized.")

# Initialize Maia engines
print("Initializing Maia engines...")
maia_engines = {}
for elo, path in maia_model_paths.items():
    maia_engines[elo] = chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"])
    print(f"Maia {elo} engine initialized.")
print("Maia engines initialized.")


Initializing Stockfish engine...


<UciProtocol (pid=19242)>: stderr >>        _
<UciProtocol (pid=19242)>: stderr >> |   _ | |
<UciProtocol (pid=19242)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19243)>: stderr >>        _
<UciProtocol (pid=19243)>: stderr >> |   _ | |
<UciProtocol (pid=19243)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19244)>: stderr >>        _
<UciProtocol (pid=19244)>: stderr >> |   _ | |
<UciProtocol (pid=19244)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19245)>: stderr >>        _
<UciProtocol (pid=19245)>: stderr >> |   _ | |
<UciProtocol (pid=19245)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19246)>: stderr >>        _
<UciProtocol (pid=19246)>: stderr >> |   _ | |
<UciProtocol (pid=19246)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19247)>: stderr >>        _
<UciProtocol (pid=19247)>: stderr >> |   _ | |
<UciProtocol (pid=19247)>: 

Stockfish engine initialized.
Initializing Maia engines...
Maia 1100 engine initialized.
Maia 1200 engine initialized.
Maia 1300 engine initialized.
Maia 1400 engine initialized.
Maia 1500 engine initialized.
Maia 1600 engine initialized.
Maia 1700 engine initialized.
Maia 1800 engine initialized.
Maia 1900 engine initialized.
Maia engines initialized.


In [24]:
# Function to load games from a PGN file
def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    print(f"Loaded {len(games)} games from PGN file {file_path}.")
    return games


In [25]:
# Function to find classic puzzles in a game
def find_classic_puzzles(game, stockfish_engine, threshold=200):
    board = game.board()
    puzzles = []

    for move in game.mainline_moves():
        fen_before = board.fen()

        # Analyze the position before the move
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1), multipv=2)
        #print(info)
        if len(info) < 2:
            continue

        best_move = info[0]["pv"][0]
        best_eval = info[0]["score"].relative.score(mate_score=10000)
        second_best_eval = info[1]["score"].relative.score(mate_score=10000)
        #print(board.is_capture(best_move))
        # Check if the difference is large enough and the move is not a simple recapture
        if abs(best_eval - second_best_eval) >= threshold and not board.is_capture(best_move):
            puzzles.append((fen_before, best_move, best_eval, second_best_eval))
            print(f"Puzzle found: {fen_before}, Best move: {best_move}, Eval difference: {abs(best_eval - second_best_eval)}")

        board.push(move)

    return puzzles

In [37]:
# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines):
    board = chess.Board(fen)
    easiness_scores = {}
    print(f"Evaluating Maia easiness for FEN: {fen}, Best Move: {best_move}")

    for elo, engine in maia_engines.items():
        result = engine.analyse(board, chess.engine.Limit(nodes=1))
        print(f"Maia {elo} evaluation: {result}")
        if "pv" in result and best_move in result["pv"]:
            print(f"Best move found in PV: {result['pv']}")
            easiness_scores[elo] = result["pv"].index(best_move) / len(result["pv"])
        else:
            easiness_scores[elo] = 0.0  # Assign 0 if the best move is not in pv

    return easiness_scores

In [27]:
# Load PGN files and find classic puzzles
pgn_directory = "analysis_pgns_test"
all_puzzles = []

print("Starting to find classic puzzles in PGN files...")
test_path = os.path.join(pgn_directory, "TFFyOff1---1908 Lasker vs. Tarrasch.pgn")
all_games = load_games_from_pgn(test_path)
for game in all_games:
    print(f"Analyzing game {game.headers.get('Event', 'Unknown Event')}")
    puzzles = find_classic_puzzles(game, stockfish_engine)
    all_puzzles.extend(puzzles)


Starting to find classic puzzles in PGN files...
Loaded 7 games from PGN file analysis_pgns_test/TFFyOff1---1908 Lasker vs. Tarrasch.pgn.
Analyzing game World Championship Match 1908
Analyzing game World Championship Match 1908
Puzzle found: 3rr3/p1pqbpkp/2pp4/5N2/4P3/1P6/P1P2KPP/R2QR3 b - - 1 17, Best move: g7h8, Eval difference: 233
Puzzle found: 3rr2k/p1pqbp1p/2pp4/5N2/3QP3/1P6/P1P2KPP/R3R3 b - - 3 18, Best move: f7f6, Eval difference: 9826
Puzzle found: 4r2k/2p4p/8/6r1/3p1p2/1P1NbQ2/P1P1R1Pq/R3K3 w - - 0 35, Best move: e1d1, Eval difference: 208
Puzzle found: 4r2k/2p4p/8/6r1/3p1p2/1P1NbQ2/P1P1R1P1/R2K2q1 w - - 2 36, Best move: d3e1, Eval difference: 260
Puzzle found: 3Q4/4r1kp/4r3/8/P2p1p2/1P2b3/2P1R1P1/R2KN1q1 b - - 0 40, Best move: f4f3, Eval difference: 243
Puzzle found: 3Q4/4r1kp/4r3/8/P2p4/1P2bP2/2P1R3/R2KN1q1 b - - 0 41, Best move: e3g5, Eval difference: 471
Analyzing game World Championship Match 1908
Puzzle found: r1bqk2r/2ppbppp/p1n2n2/1p2p3/B3P3/5N2/PPPP1PPP/RNBQR1K1 w kq

In [28]:
puzzles

[('3r4/p3kp1p/5p2/1Rn1p3/8/2P1P1N1/P1P2RPP/3r2K1 w - - 5 24',
  Move.from_uci('f2f1'),
  70,
  -185)]

In [38]:
for fen, best_move, best_eval, second_best_eval in puzzles:
    easiness_scores = evaluate_maia_easiness(fen, best_move, maia_engines)
    #print(easiness_scores)

Evaluating Maia easiness for FEN: 3r4/p3kp1p/5p2/1Rn1p3/8/2P1P1N1/P1P2RPP/3r2K1 w - - 5 24, Best Move: f2f1
Maia 1100 evaluation: {'depth': 1, 'seldepth': 0, 'time': 0.0, 'nodes': 1, 'score': PovScore(Cp(-32), WHITE), 'tbhits': 0, 'pv': [Move.from_uci('g3f1')]}
Maia 1200 evaluation: {'depth': 1, 'seldepth': 0, 'time': 0.0, 'nodes': 1, 'score': PovScore(Cp(-12), WHITE), 'tbhits': 0, 'pv': [Move.from_uci('f2f1')]}
Best move found in PV: [Move.from_uci('f2f1')]
Maia 1300 evaluation: {'depth': 1, 'seldepth': 0, 'time': 0.0, 'nodes': 1, 'score': PovScore(Cp(-264), WHITE), 'tbhits': 0, 'pv': [Move.from_uci('f2f1')]}
Best move found in PV: [Move.from_uci('f2f1')]
Maia 1400 evaluation: {'depth': 1, 'seldepth': 0, 'time': 0.0, 'nodes': 1, 'score': PovScore(Cp(+6), WHITE), 'tbhits': 0, 'pv': [Move.from_uci('g3f1')]}
Maia 1500 evaluation: {'depth': 1, 'seldepth': 0, 'time': 0.0, 'nodes': 1, 'score': PovScore(Cp(+7), WHITE), 'tbhits': 0, 'pv': [Move.from_uci('f2f1')]}
Best move found in PV: [Move.